In [44]:
import numpy as np
import pandas as pd

In [57]:
# File name and Route
rv_route = '/Users/myeongseop.kim/Desktop/SCAISCO/csv files/'
tor_route = '/Users/myeongseop.kim/Desktop/SCAISCO/turn_over_rate/'
fv_route = '/Users/myeongseop.kim/Desktop/SCAISCO/financial_variable/'

rv_name = ['bank_financial_business_review.csv',
'construction_review.csv',
'distribution_trade_transport_reviews.csv',
'education_review.csv',
'it_web_communication_reviews_101_200.csv',
'it_web_communication_reviews_1_100.csv',
'it_web_communication_reviews_201_300.csv',
'it_web_communication_reviews_300_end.csv',
'manufacture_chemistry_review.csv',
'media_design_reviews.csv',
'medical_medicine_welfare_reviews.csv',
'organization_association_reviews.csv',
'service_reviews.csv']

tor_name = [
    'bank_financial_business_turn_over_rate.csv',
    'construction_turn_over_rate.csv',
    'distribution_trade_transport_turn_over_rate.csv',
    'education_turn_over_rate.csv',
    'it_web_communication_101_200_turn_over_rate.csv',
    'it_web_communication_1_100_turn_over_rate.csv',
    'it_web_communication_201_300_turn_over_rate.csv',
    'it_web_communication_300_end_turn_over_rate.csv',
    'manufacture_chemistry_turn_over_rate.csv',
    'media_design_turn_over_rate.csv',
    'medical_medicine_welfare_turn_over_rate.csv',
    'organization_association_turn_over_rate.csv',
    'service_turn_over_rate.csv'
]

fv_name = [
    'bank_financial_financial_variable.csv',
    'construction_financial_variable.csv',
    'distribution_trade_transport_financial_variable.csv',
    'education_financial_variable.csv',
    'it_web_communication_101_200_financial_variable.csv',
    'it_web_communication_1_100_financial_variable.csv',
    'it_web_communication_201_300_financial_variable.csv',
    'it_web_communication_300_end_financial_variable.csv',
    'manufacture_chemistry_financial_variable.csv',
    'media_design_financial_variable.csv',
    'medical_medicine_welfare_financial_variable.csv',
    'organization_association_financial_variable.csv',
    'service_financial_variable.csv'
]


In [58]:
print(len(fv_name))
print(len(tor_name))
print(len(rv_name))

13
13
13


In [59]:
def rv_preprocessing(rv):
    rv_df = rv
    cn = rv_df['cmp']
    
    for i, val in enumerate(cn):
        if '(주)' in val:
            tmp = val.replace('(주)','')
            rv_df.loc[i,'cmp'] = tmp
    
    col_list = list(rv_df.columns)
    col_list[0] = 'company_name'
    rv_df.columns = col_list

    return rv_df

In [60]:
# Preprocessing for financial variable

def fv_preprocessing(fv):
  fv_df = fv
  afv = fv_df['average_salary']
  tfv = fv_df['total_sale']

  for i, val in enumerate(afv):
    if '만원' in val:
      tmp = int(val[:-2].replace(',',''))
      fv_df.loc[i,'average_salary'] = tmp
    if '회사' in val:
      fv_df.loc[i,'average_salary'] = np.NAN
    if '수집' in val:
      fv_df.loc[i,'average_salary'] = np.NAN


  for i, val in enumerate(tfv):
    if '조원' in val:
      tmp = float(val[:-2].replace(',',''))*1000
      fv_df.loc[i,'total_sale'] = tmp
    if '억원' in val:
      tmp = float(val[:-2].replace(',',''))
      fv_df.loc[i,'total_sale'] = tmp
    if '회사' in val:
      fv_df.loc[i,'total_sale'] = np.NAN


  
  return fv_df

In [61]:
# Preprocessing for turn over rate
def tor_preprocessing(tor):
  tor_df = tor
  tor_val = tor_df['turn_over_rate']

  for i, val in enumerate(tor_val):
    if '정보' in val:
      tor_df.loc[i,'turn_over_rate'] = np.NAN
    elif val == '(9999%)' :
      tor_df.loc[i,'turn_over_rate'] = np.NAN
    elif '(' in val:
      tmp = float(val[1:-1].replace('%',''))*0.01
      tor_df.loc[i,'turn_over_rate'] = tmp

  return tor_df

In [65]:
# BOW DF

def bow_categorical_amalgation(num):

    rv_tmp = pd.read_csv(rv_route+rv_name[num])
    fv_tmp = pd.read_csv(fv_route+fv_name[num])
    tor_tmp = pd.read_csv(tor_route+tor_name[num])

    rv_tmp = rv_tmp.drop(['Unnamed: 0'], axis=1)
    fv_tmp = fv_tmp.drop(['Unnamed: 0'], axis=1)
    tor_tmp = tor_tmp.drop(['Unnamed: 0'], axis=1)

    rv_tmp = rv_preprocessing(rv_tmp)
    fv_tmp = fv_preprocessing(fv_tmp)
    tor_tmp = tor_preprocessing(tor_tmp)

    bow_df = pd.merge(rv_tmp, fv_tmp, on = 'company_name', how = 'inner')
    bow_df = pd.merge(bow_df, tor_tmp, on = 'company_name', how = 'inner')
    bow_df = bow_df.dropna()
    return bow_df

for i in range(len(rv_name)):
    if i == 0:
        bow_df = bow_categorical_amalgation(i)
    else:
        bow_df = pd.concat([bow_df,bow_categorical_amalgation(i)])

bow_df.to_csv("bow_df.csv")


In [ ]:
# Topic Modeling DF
